In [1]:
import pandas
from create_train_test_val_maps import *

%load_ext autoreload
%autoreload 2

In [2]:
s_to_date = lambda x: pandas.to_datetime(x, format='%m/%d/%Y %H:%M:%S')
snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_full_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

s_to_date = lambda x: pandas.to_datetime(x, format='%Y/%m/%d %H:%M:%S')
ok_snapshots = pandas.read_csv('/home/cs231n/data/selected_snapshots_ok_deduped_backfilled_whitened.csv', converters={'Event DateTime':s_to_date})

In [3]:
ata6_train_repairs = pandas.read_csv('ATA6_train.csv')
ata6_val_repairs = pandas.read_csv('ATA6_val.csv')
ata6_test_repairs = pandas.read_csv('ATA6_test.csv')

category_var = ['Event Type Description','Brake Switch','Clutch Switch','Cruise Status','Dpf Regen Inhibit Sw', 
                'Dpf Thermal Mngmnt','Eng Coolant Level','DTCID']
for cat in category_var:
    snapshots[cat] = snapshots[cat].astype('category').cat.codes
    ok_snapshots[cat] = ok_snapshots[cat].astype('category').cat.codes

try:
    snapshots = snapshots.drop(['DTC'],1)
except:
    pass
try:
    snapshots = snapshots.drop(['Unnamed: 0'],1)
except:
    pass
try:
    snapshots = snapshots.drop(['Ignition Cycle Counter'],1)
except:
    pass
    
try:
    ok_snapshots = ok_snapshots.drop(['DTC'],1)
except:
    pass
try:
    ok_snapshots = ok_snapshots.drop(['Unnamed: 0'],1)
except:
    pass
try:
    ok_snapshots = ok_snapshots.drop(['Ignition Cycle Counter'],1)
except:
    pass

train_veh_ids = ata6_train_repairs['Chassis Reference Number'].unique().astype(int)
val_veh_ids = ata6_val_repairs['Chassis Reference Number'].unique().astype(int)
test_veh_ids = ata6_test_repairs['Chassis Reference Number'].unique().astype(int)

all_ok_veh_ids = ok_snapshots['Veh Ref ID'].unique().astype(int)
# these are carefully tuned proportions to match lowest frequency code in each group
train_ok_ids = all_ok_veh_ids[0:5]
val_ok_ids = all_ok_veh_ids[5:8]
test_ok_ids = all_ok_veh_ids[8:11]

ids_tuple = (train_veh_ids, val_veh_ids, test_veh_ids)
ids_ok_tuple = (train_ok_ids, val_ok_ids, test_ok_ids)
repairs_tuple = (ata6_train_repairs, ata6_val_repairs, ata6_test_repairs)

In [ ]:
# run this first in your terminal to clear out previous maps
# > cd /home/cs231n/data; sudo find . -name "train*" -delete; sudo find . -name "val*" -delete; sudo find . -name "test*" -delete;

In [26]:
hyper_tuples = [(1,500), (2,250), (5,100), (10, 50)]

for num_windows, window_size in hyper_tuples:
    create_maps(num_windows, window_size, snapshots, ok_snapshots, ids_tuple, ids_ok_tuple, repairs_tuple, incomplete_windows=True)

In [ ]:
train_combined, val_combined, test_combined = load_maps(10,50)

In [ ]:
# only run this if you are readjusting the proportions
# num_windows, window_size = (5,100)
# ok_slices = get_ok_slices_map(train_ok_ids, ok_snapshots, 1, window_size, ignore_past=True)
# print(sum([len(ok_slices[i][0][0][0]) for i in ok_slices.keys()]))
# repair_slices = get_repair_slices_map(train_veh_ids, snapshots, ata6_train_repairs, num_windows, window_size, code='ATA6', ignore_past=True)
# print(sum([len(repair_slices[i][45008][0][0]) for i in repair_slices.keys() if 45008 in repair_slices[i]]))

# num_windows, window_size = (10,10)
# ok_slices = get_ok_slices_map(val_ok_ids, ok_snapshots, 1, window_size, ignore_past=True)
# print(sum([len(ok_slices[i][0][0][0]) for i in ok_slices.keys()]))
# repair_slices = get_repair_slices_map(val_veh_ids, snapshots, ata6_val_repairs, num_windows, window_size, code='ATA6', ignore_past=True)
# print(sum([len(repair_slices[i][45007][0][0]) for i in repair_slices.keys() if 45007 in repair_slices[i]]))

# num_windows, window_size = (10,10)
# ok_slices = get_ok_slices_map(test_ok_ids, ok_snapshots, 1, window_size, ignore_past=True)
# print(sum([len(ok_slices[i][0][0][0]) for i in ok_slices.keys()]))
# repair_slices = get_repair_slices_map(test_veh_ids, snapshots, ata6_test_repairs, num_windows, window_size, code='ATA6', ignore_past=True)
# print(sum([len(repair_slices[i][43004][0][0]) for i in repair_slices.keys() if 43004 in repair_slices[i]]))